In [1]:
# Import the main libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# Import the datasets with the coformers and their descriptors
dataset1 = pd.read_csv('/content/drive/My Drive/cocrystal_design/data/coformers1.csv')
dataset2 = pd.read_csv('/content/drive/My Drive/cocrystal_design/data/coformers2.csv')
dataset1.head()

,Identifier,NAME,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,nAT,nSK,nBT,nBO,nBM,SCBO,RBN,RBF,nDB,nTB,nAB,nH,nC,nN,nO,nP,nS,nF,nCL,nBR,nHM,nHet,nX,H%,C%,N%,O%,X%,...,Ui,Hy,AMR,TPSA(NO),TPSA(Tot),MLOGP,MLOGP2,ALOGP,ALOGP2,SAtot,SAacc,Vx,VvdwMG,VvdwZAZ,PDI,BLTF96,BLTD48,BLTA96,cRo5,DLS_01,DLS_02,DLS_03,DLS_04,DLS_05,DLS_06,DLS_cons,LLS_01,LLS_02,CMC-80,CMC-50,Inflammat-80,Inflammat-50,Depressant-80,Psychotic-80,Hypertens-80,Hypnotic-80,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,WOVMEB,Cc1ccccc1C,106.18,5.899,10.634,17.418,11.807,20.076,0.591,0.968,0.656,1.115,18,8,18,8,6,11.0,0,0.000,0,0,6,10,8,0,0,0,0,0,0,0,0,0,0,55.6,44.4,0.0,0.0,0.0,...,2.000,-0.946,36.140,0.00,0.00,2.942,8.653,2.802,7.853,179.638,0.000,165.814,71.334,115.78,0.923,-3.89,-4.11,-4.17,1,1.00,0.50,0.83,0.7,0.0,1.00,0.72,1.00,0.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PIWKEL,O=C1C=CC2=C(C=C1)C(=O)C1=C(C=CC(=O)C=C1)C2=O,264.24,9.437,20.966,28.844,20.864,30.498,0.749,1.030,0.745,1.089,28,20,30,22,10,32.0,0,0.000,10,0,0,8,16,0,4,0,0,0,0,0,0,4,0,28.6,57.1,0.0,14.3,0.0,...,3.459,-0.787,78.415,68.28,68.28,1.749,3.058,0.969,0.939,303.835,100.580,305.980,128.779,257.04,1.007,-2.88,-2.98,-2.98,1,1.00,0.83,1.00,0.4,0.0,1.00,0.75,0.67,1.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
2,OTIHIJ,CN(C)c1ccc(cc1)C(C(C1=CC=C2C=CC=CC=C12)C1=C2C=...,520.76,6.852,48.998,74.225,52.955,84.055,0.645,0.977,0.697,1.106,76,40,81,45,34,62.0,7,0.086,0,0,34,36,38,2,0,0,0,0,0,0,0,2,0,47.4,50.0,2.6,0.0,0.0,...,4.170,-0.944,171.441,6.48,6.48,6.906,47.688,8.970,80.461,700.619,6.249,718.073,297.669,506.16,1.025,-7.26,-7.88,-8.14,0,0.50,0.50,0.67,0.4,0.5,0.67,0.60,0.50,0.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DABLOJ,CCOC(C)=O,88.12,6.294,7.537,14.189,7.955,16.080,0.538,1.014,0.568,1.149,14,6,13,5,1,6.0,2,0.154,1,0,0,8,4,0,2,0,0,0,0,0,0,2,0,57.1,28.6,0.0,14.3,0.0,...,1.000,-0.614,22.161,26.30,26.30,0.586,0.344,0.370,0.137,159.748,36.145,124.020,54.205,92.70,0.776,-1.89,-1.88,-1.82,1,1.00,0.83,0.83,1.0,1.0,1.00,0.95,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PIGJUK,CSC1=Cc2ccc3SC(=Cc4ccc(S1)c2c34)SC,332.56,10.393,23.909,31.607,27.159,34.171,0.747,0.988,0.849,1.068,32,20,35,23,13,30.5,2,0.057,2,0,11,12,16,0,0,0,4,0,0,0,4,4,0,37.5,50.0,0.0,0.0,0.0,...,3.087,-0.787,99.705,0.00,107.08,5.373,28.870,5.646,31.875,355.988,0.000,379.003,158.706,269.68,1.065,-5.96,-6.42,-6.60,1,0.75,0.67,0.83,0.2,0.0,0.83,0.61,0.50,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Construct the labelled dataset by contatenating the feature set of each coformer in both orders
def concat_bidirectional(dataset11, dataset22):

  return pd.concat([pd.concat([dataset1['Identifier'], dataset11, dataset22], axis=1), pd.concat([dataset1['Identifier'].apply(lambda x: f"{x}_"),dataset22, dataset11], axis=1) ])


In [6]:
# Standarize the dataset

X_scaler = MinMaxScaler()
df1=dataset1.iloc[:,2:]
df1 = df1.fillna(0)
df2=dataset2.iloc[:,2:]
df2 = df2.fillna(0)
df1=df1.dropna(axis=1)
df2=df2[df1.columns.values]
df_concat = pd.concat([df1, df2])
df_concat = df_concat.drop_duplicates(keep='first')
numerical_cols = df_concat.columns[:]
df_scaled = pd.DataFrame(X_scaler.fit(df_concat), columns=numerical_cols, index=df_concat.index)

numerical_cols = df2.columns[:]
df1_scaled =  pd.DataFrame(X_scaler.transform(df1[numerical_cols]), columns=numerical_cols, index=df1.index)
df2_scaled = pd.DataFrame(X_scaler.transform(df2[numerical_cols]), columns=numerical_cols, index=df2.index)

# Final bidirectional concatenated dataset, after feature selection and scaling 
df = concat_bidirectional(df1_scaled,df2_scaled)
df

,Identifier,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,nAT,nSK,nBT,nBO,nBM,SCBO,RBN,RBF,nDB,nTB,nAB,nH,nC,nN,nO,nP,nS,nF,nCL,nBR,nHM,nHet,nX,H%,C%,N%,O%,X%,nCsp3,...,Ui,Hy,AMR,TPSA(NO),TPSA(Tot),MLOGP,MLOGP2,ALOGP,ALOGP2,SAtot,SAacc,Vx,VvdwMG,VvdwZAZ,PDI,BLTF96,BLTD48,BLTA96,cRo5,DLS_01,DLS_02,DLS_03,DLS_04,DLS_05,DLS_06,DLS_cons,LLS_01,LLS_02,CMC-80,CMC-50,Inflammat-80,Inflammat-50,Depressant-80,Psychotic-80,Hypertens-80,Hypnotic-80,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,WOVMEB,0.012474,0.041630,0.045011,0.018627,0.017335,0.021339,0.558074,0.215152,0.125156,0.408922,0.021390,0.016043,0.023018,0.013825,0.034483,0.019737,0.000000,0.000000,0.0000,0.0,0.034483,0.047619,0.031579,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.794286,0.551977,0.000000,0.000000,0.0,0.020833,...,0.766992,0.085463,0.449801,0.000000,0.846523,0.725989,0.273423,0.585007,0.304193,0.000000,0.000000,0.497439,0.497439,0.408947,0.000000,0.383961,0.384058,0.384165,0.0,0.333333,0.192771,0.507463,0.5,0.0,0.39759,0.411765,0.192771,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PIWKEL,0.070361,0.134320,0.088745,0.050028,0.053788,0.046290,0.707271,0.403030,0.236546,0.312268,0.048128,0.080214,0.053708,0.078341,0.057471,0.088816,0.000000,0.000000,0.6250,0.0,0.000000,0.038095,0.073684,0.000000,0.148148,0.0,0.00,0.0,0.0,0.0,0.000000,0.088889,0.0,0.408571,0.737921,0.000000,0.333333,0.0,0.000000,...,0.309645,0.120102,0.066373,0.000000,0.211631,0.484566,0.010521,0.102856,0.000931,0.058065,0.000000,0.032268,0.032267,0.028574,0.704039,0.722560,0.722464,0.722547,1.0,1.000000,0.602410,1.000000,0.3,0.0,1.00000,0.658824,0.795181,0.840000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,OTIHIJ,0.164308,0.066597,0.207398,0.174745,0.182948,0.174511,0.609065,0.242424,0.176471,0.375465,0.176471,0.187166,0.184143,0.184332,0.195402,0.187500,0.145833,0.362869,0.0000,0.0,0.195402,0.171429,0.189474,0.133333,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.044444,0.0,0.677143,0.633968,0.045534,0.000000,0.0,0.062500,...,0.309645,0.022147,0.031345,0.000000,0.000000,0.481029,0.009296,0.105014,0.001078,0.035703,0.000000,0.004748,0.004748,0.002510,0.649721,0.727420,0.727536,0.727711,1.0,1.000000,0.397590,0.746269,0.5,0.0,1.00000,0.635294,1.000000,0.840000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DABLOJ,0.005860,0.051978,0.031903,0.009753,0.001831,0.011772,0.508026,0.354545,0.015019,0.535316,0.010695,0.005348,0.010230,0.000000,0.005747,0.003289,0.041667,0.649789,0.0625,0.0,0.000000,0.038095,0.010526,0.000000,0.074074,0.0,0.00,0.0,0.0,0.0,0.000000,0.044444,0.0,0.815714,0.320644,0.000000,0.333333,0.0,0.031250,...,0.651649,0.038898,0.185238,0.323269,0.184353,0.511972,0.022585,0.183961,0.013957,0.204266,0.326708,0.175069,0.175069,0.155583,0.649025,0.684083,0.684058,0.684337,1.0,1.000000,0.795181,1.000000,0.8,0.5,1.00000,0.858824,0.000000,0.840000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PIGJUK,0.095382,0.159366,0.101202,0.057622,0.079124,0.055084,0.705382,0.275758,0.366708,0.234201,0.058824,0.080214,0.066496,0.082949,0.074713,0.083882,0.041667,0.240506,0.1250,0.0,0.063218,0.057143,0.073684,0.000000,0.000000,0.0,0.25,0.0,0.0,0.0,0.129032,0.088889,0.0,0.535714,0.633968,0.000000,0.000000,0.0,0.020833,...,0.604892,0.060193,0.095491,0.312625,0.178282,0.442278,0.000837,0.104051,0.001011,0.094449,0.318692,0.066640,0.066640,0.069483,0.666435,0.781693,0.781884,0.781756,1.0,1.000000,0.795181,1.000000,0.4,0.0,1.00000,0.705882,0.602410,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,AFOTIB_,0.007336,0.048022,0.038548,0.010702,0.010247,0.013163,0.573182,0.218182,0.142678,0.394052,0.013369,0.010695,0.015345,0.009217,0.034483,0.016447,

In [8]:
# Generate the unknown dataset (unlabelled)
# Read the Zinc dataset of purcasable molecules with their dragon descriptors
unlabeled = pd.read_csv('/content/drive/My Drive/cocrystal_design/data/unlabelled_dragon_descriptors.csv')

# Take all the possible combinations between these molecules
val = unlabeled['NAME'].values
length = len(val)
unlabeled=unlabeled.loc[:, (unlabeled != 0).any(axis=0)]
pairs = [[val[i],val[j]] for i in range(length) for j in range(length) if i!=j ]

# Remove the duplicate structures
no_dups = []
for pair in pairs:
  if not any(all(i in p for i in pair) for p in no_dups):
    no_dups.append(pair)

pairs = pd.DataFrame(no_dups)

# Assign to each molecule its descriptor vector
keys = unlabeled['NAME'].values
values = unlabeled.iloc[:, 1:].values
d = {key:value for key, value in zip(keys, values)}


In [9]:
# Construct two datasets of the co-former pairs extracted from ZINC15
mol1_data= list()
for mol1 in pairs[0]:       
    mol1_data.append(d[mol1])    
mol1_data = pd.DataFrame(mol1_data, columns = unlabeled.iloc[:, 1:].columns.values)   
mol2_data= list()
for mol2 in pairs[1]:   
    mol2_data.append(d[mol2])
mol2_data = pd.DataFrame(mol2_data, columns= unlabeled.iloc[:, 1:].columns.values) 

final_1= mol1_data.iloc[:,1:][dataset1.columns.values[2:]]
final_2= mol2_data.iloc[:,1:][dataset1.columns.values[2:]]

In [11]:
# Save the datasets
final_1.to_csv('/content/drive/My Drive/cocrystal_design/data/unlabeled1.csv', index=False)
final_2.to_csv('/content/drive/My Drive/cocrystal_design/data/unlabeled2.csv', index=False)

In [ ]:
# Standarize the unlabeled data based on the labelled
final_1_scaled = pd.DataFrame(X_scaler.transform(final_1))
final_2_scaled = pd.DataFrame(X_scaler.transform(final_2))

uf=pd.concat([final_1_scaled, final_2_scaled], axis =1)

In [ ]:
# Construct the final dataset by adding the combination id and the scaled descroiptors
uf_final = pd.concat([uf_tot['comb'], pd.DataFrame(uf)], axis=1)
uf_final = pd.DataFrame(uf_final.values , columns=df.columns.values )
uf_final

,Identifier,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,nAT,nSK,nBT,nBO,nBM,SCBO,RBN,RBF,nDB,nTB,nAB,nH,nC,nN,nO,nP,nS,nF,nCL,nBR,nHM,nHet,nX,H%,C%,N%,O%,X%,nCsp3,...,Ui,Hy,AMR,TPSA(NO),TPSA(Tot),MLOGP,MLOGP2,ALOGP,ALOGP2,SAtot,SAacc,Vx,VvdwMG,VvdwZAZ,PDI,BLTF96,BLTD48,BLTA96,cRo5,DLS_01,DLS_02,DLS_03,DLS_04,DLS_05,DLS_06,DLS_cons,LLS_01,LLS_02,CMC-80,CMC-50,Inflammat-80,Inflammat-50,Depressant-80,Psychotic-80,Hypertens-80,Hypnotic-80,Neoplastic-80,Neoplastic-50,Infective-80,Infective-50
0,comb1,0.0586417,0.12109,0.0826959,0.0427319,0.0501292,0.040501,0.70916,0.333333,0.26408,0.27881,0.0427807,0.0695187,0.0511509,0.0737327,0.0862069,0.0838816,0,0,0.25,0,0.0632184,0.0380952,0.0736842,0,0.0740741,0,0,0,0,0,0,0.0444444,0,0.44,0.802343,0,0.179487,0,0,...,0.555039,0.0048268,0.089599,0,0,0.552491,0.0487646,0.145094,0.00579704,0.0652703,0,0.0551251,0.055125,0.0379611,0.850975,0.62738,0.627174,0.627539,1,0.666667,0.39759,0.746269,0.2,0,0.795181,0.517647,0.795181,0.506667,1,0,1,0,0,0,1,0,1,0,1,0
1,comb2,0.0586417,0.12109,0.0826959,0.0427319,0.0501292,0.040501,0.70916,0.333333,0.26408,0.27881,0.0427807,0.0695187,0.0511509,0.0737327,0.0862069,0.0838816,0,0,0.25,0,0.0632184,0.0380952,0.0736842,0,0.0740741,0,0,0,0,0,0,0.0444444,0,0.44,0.802343,0,0.179487,0,0,...,0.572844,0.00425894,0.100047,0,0,0.56197,0.0563241,0.155222,0.00758334,0.0771716,0,0.0671745,0.0671741,0.0494382,0.820334,0.614014,0.61413,0.614114,1,0.666667,0.39759,0.746269,0.2,0,0.795181,0.517647,0.795181,0.506667,1,0,0,0,1,0,0,0,0,0,1,0
2,comb3,0.0586417,0.12109,0.0826959,0.0427319,0.0501292,0.040501,0.70916,0.333333,0.26408,0.27881,0.0427807,0.0695187,0.0511509,0.0737327,0.0862069,0.0838816,0,0,0.25,0,0.0632184,0.0380952,0.0736842,0,0.0740741,0,0,0,0,0,0,0.0444444,0,0.44,0.802343,0,0.179487,0,0,...,0.597306,0.00369108,0.109386,0,0,0.571032,0.0640682,0.160169,0.00854458,0.0786128,0,0.0737442,0.0737438,0.0525624,0.858635,0.601458,0.601449,0.601377,1,0.666667,0.192771,0.746269,0.2,0,0.795181,0.482353,0.795181,0.506667,1,0,0,0,0,0,0,0,0,0,0,0
3,comb4,0.0586417,0.12109,0.0826959,0.0427319,0.0501292,0.040501,0.70916,0.333333,0.26408,0.27881,0.0427807,0.0695187,0.0511509,0.0737327,0.0862069,0.0838816,0,0,0.25,0,0.0632184,0.0380952,0.0736842,0,0.0740741,0,0,0,0,0,0,0.0444444,0,0.44,0.802343,0,0.179487,0,0,...,0.619291,0.00340716,0.118726,0,0,0.57975,0.0719787,0.165117,0.00956312,0.080054,0,0.0803138,0.0803135,0.0556866,0.89624,0.589307,0.58913,0.589329,1,0.666667,0.192771,0.746269,0.2,0,0.795181,0.482353,0.60241,0.506667,1,0,0,0,0,0,0,0,0,0,0,0
4,comb5,0.0586417,0.12109,0.0826959,0.0427319,0.0501292,0.040501,0.70916,0.333333,0.26408,0.27881,0.0427807,0.0695187,0.0511509,0.0737327,0.0862069,0.0838816,0,0,0.25,0,0.0632184,0.0380952,0.0736842,0,0.0740741,0,0,0,0,0,0,0.0444444,0,0.44,0.802343,0,0.179487,0,0,...,0.45007,0.00851789,0.0624356,0,0,0.520837,0.0274646,0.122696,0.00269732,0.0508301,0,0.0299367,0.0299364,0.0241499,0.77507,0.671932,0.671739,0.671601,1,1,0.39759,0.746269,0.5,0,1,0.635294,0.795181,0.84,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21731,comb21732,0.0744301,0.0874771,0.102357,0.0690736,0.0721047,0.0688113,0.634561,0.293939,0.187735,0.3829,0.0695187,0.0855615,0.0767263,0.0875576,0.12069,0.0970395,0.0208333,0.109705,0,0,0.12069,0.0714286,0.0842105,0.133333,0.037037,0,0,0,0,0,0,0.0666667,0,0.595714,0.633968,0.0980736,0.0652681,0,0.0208333,...,0.581359,0.00425894,0.0989382,0,0,0.56197,0.0563241,0.150042,0.0066411,0.0667115,0,0.0616948,0.0616947,0.0410854,0.89624,0.614014,0.61413,0.614114,1,0.666667,0.192771,0.746269,0.2,0,0.795181,0.482353,0.795181,0.506667,1,0,0,0,0,0,0,0,1,0,1,0
21732,comb21733,0.0744301,0.0874771,0.102357,0.0690736,0.0721047,0.0688113,0.634561,0.293939,0.187735,0